# Introduction to Conventions

Adhering to **metadata standards** is essential to achieve **findability** and efficient **re-usability** of data. Typos and attribute naming based on personal preferences must be avoided. Even if humans may still be able to understand such data, especially automatic exploration of meta information and processing of it will be impossible.

The `h5RDMtoolbox` solves this by introducing the **conventions**. A convention is a selection of **standard_attributes** which clearly define how metadata need to be provided and for which HDF5 objects (datasets or groups).

It is written in a YAML or JSON file and is defined by data mangers (stakeholders). Please note, that it is up those, who are defining the standard attributes, whether the standard is good scientific practice and fulfills the requirements of FAIR metadata. The toolbox just implements the rules in the interface between data creator and the HDF5 file.

## 1. Learning from a simple practical example
To understand the concept of a convention easier, let's start by examining a simple example:

Let's decide, that the work with scientific data must include its physical unit in the HDF5 datasets. So, as a data manager, we can define our requirement like so:

    Each dataset in an HDF5 file must have the attribute "units"

Concretely, this means, that we expect each dataset to receiv an attribute which is called "units". Furthermore, we expect that the value of it is reasonable, thus convertable to SI-units.

Consider the following code. It shows the creation of an HDF5 dataset. The user might only have heard of the rules but was not quite strikt adhering to it:

In [1]:
import h5rdmtoolbox as h5tbx

with h5tbx.File() as h5:
    ds = h5.create_dataset('velocity', data=[1, -4, 3])
    ds.attrs['unit'] = 'meter_per_second'

non_compliant_filename = h5.hdf_filename

The creator of the dataset tried to comply with the above requirement by adding a unit attribute to the dataset. However, a spelling error occurred ("unit" instead of "units").

Also, with a bit of experience, we can already guess that the value for unit is not easily processed by further (Python) processing. Using "m/s" would have been the better choice (for various reasons not outlined here for the sake of simplicity).

### 1.1 Use a convention

Now, without knowing how conventions are designed and work yet, let's load a convention definition file (a [YAML](https://pyyaml.org/wiki/PyYAMLDocumentation) file). It includes the above rule for the usage of units.

We activate the convention by calling `use()`:

In [2]:
cv = h5tbx.convention.from_yaml('units-convention.yaml')
h5tbx.use(cv)  # or if we know the name: h5tbx.use("tutorial-units-convention")

using("tutorial-units-convention")

It turns out, that the `create_dataset` now requires an additional parameter, namely "units". This was defined in the YAML file (we take a look into the file later):

In [3]:
with h5tbx.File() as h5:
    try:
        ds = h5.create_dataset('velocity', data=[1, -4, 3])
    except h5tbx.errors.StandardAttributeError as e:
        print(e)

Convention "tutorial-units-convention" expects standard attribute "units" to be provided as an argument during dataset creation.


As a result, the code needs to be fixed:

In [4]:
with h5tbx.File() as h5:
    ds = h5.create_dataset('velocity', data=[1, -4, 3], units='m/s')

In [5]:
with h5tbx.use(None):
    with h5tbx.File() as h5:
        ds = h5.create_dataset('velocity', data=[1, -4, 3], attrs=dict(units='invalid_unit'))

In [6]:
with h5tbx.File(h5.hdf_filename) as h5:
    print(h5.velocity.units)

invalid_unit


C:\Users\da4323\Documents\programming\GitHub\h5RDMtoolbox\h5rdmtoolbox\convention\standard_attributes.py:334: StandardAttributeValidationWarning: The attribute "units" could not be validated by the convention "tutorial-units-convention".
Pydantic error: 
1 validation error for units
value
  Value error, Units cannot be understood using ureg package: invalid_unit. Original error: 'invalid_unit' is not defined in the unit registry [type=value_error, input_value='invalid_unit', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/value_error
  warnings.warn(f'The attribute "{self.name}" could not be validated by the convention '


### 1.2 Validation
The above is direct feedback when creating HDF5 files and using the `h5RDMtoolbox` and therefore helpful for developers or data creators.

But what if we have a file from another source that is in compliance with our convention? Then we can call `.validate()` from our convention and pass the filename of the file that we want to validate. We will get a list of dictionary entries, which tell us which attributes are invalid and why:

In [7]:
cv.validate(non_compliant_filename)

[{'name': '/velocity', 'attr_name': 'units', 'reason': 'missing_attribute'}]

## Creating a convention (file)

A convention is defined in a YAML (or JSON) file and consists of three parts:

    1. General information (keywords must start and end with double underscores)
    2. Definition of *standard attributes*
    3. Definition of *standard attribute validators*

1. General information at the header indicated by double underscores:

2. Definition of standard attributes

A `standard_attribute` has various properties like the `description`, `target_method` (during which the standard attribute can be passed), the `validator`, which validates the input and the `default_value`. The latter can be "\\$EMPTY" indicating that no default value is set, and thus this attribute is obligatory. "\\$NONE" indicates, that the attribute is optional. And finally, a valid value can be given to be written even if no input is provided for this standard attribute.

3. Special type definitions

Here the allowed values for the standard attribute `data_type` is listed:

The heart of standard attributes is the **validator**. A validator becomes effective when metadata is written.
The Flow chart below illustrates the writing and reading procedure for the example of writing the attribute "units".

1. Writing (`ds.units = 'm/s'`): If "units" is defined in the convention, the validator checks the value. "m/s" is a correct unit, so it will be written. Otherwise, invalid values raise an error.
3. Reading (`print(ds.units)`): The validator becomes effective upon reading attributes, that are standardized. However, then invalid value only raise warnings, in order to allow the user to still work with the file and fix the issue.

<img src="../_static/h5RDMtoolbox_standard_attribute_concept.png" alt="h5RDMtoolbox_standard_attribute_concept.png" width="800"/>

## Reading a Convention from a file:

Let's read the above example file into the class `Convention`. The object representation displays the standard attributes which are expected for the root group (`File.__init__()`), group creation (`Group.create_group()`) and dataset creation (`Group.create_dataset()`).

Note, that the standard attributes, which are marked **bold**, are obligatory. The others may or may not be provided during object creation:

In [8]:
h5tbx.convention.utils.yaml2json('example_convention.yaml')

WindowsPath('example_convention.json')

In [9]:
cv = h5tbx.convention.from_yaml('example_convention.yaml')
# you may also use .from_json('example_convention.json')
cv

Convention("h5rdmtoolbox-tuturial-convention")
contact: https://orcid.org/0000-0001-8729-0482
  File.__init__():
    * contact_id:
		ID of a person to contact for questions.
    * data_type (default=None):
		Type of data in file. Can be numerical or experimental.
  Group.create_dataset():
    * units:
		The physical unit of the dataset. If dimensionless, the unit is ''.
    * comment (default=None):
		Comment to a dataset.
  Group.create_group():
    * comment (default=None):
		Comment to a dataset.

In order to make the convention affective in this session, it must be enabled. We do this by calling `use()`:

In [10]:
h5tbx.use(cv)

using("h5rdmtoolbox-tuturial-convention")

Now, we will get an error if we create a HDF5 file without providing the attribute `contact_id`. As we made it a required attribute, it must be provided during file initialization:

In [11]:
try:
    with h5tbx.File() as h5:
        pass
except h5tbx.errors.StandardAttributeError as e:
    print(e)

Convention "h5rdmtoolbox-tuturial-convention" expects standard attribute "contact_id" to be provided as an argument during file creation.


Providing a wrong value raises an error, too:

In [12]:
try:
    with h5tbx.File(contact_id='id1722') as h5:
        h5.create_dataset(name='velocity', shape=(3, 4), units='m/s', comment='velocity field')
except h5tbx.errors.StandardAttributeError as e:
    print(e)

The value "velocity field" for standard attribute "comment" could not be set. Please check the convention file wrt. the rule for this attribute. The following error message might not always explain the origin of the problem:
1 validation error for comment
value
  Value error, Invalid format for pattern [type=value_error, input_value='velocity field', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/value_error


Now, we got it:

In [13]:
with h5tbx.File(contact_id='id1722') as h5:
    h5.dump()

Note, that if we were to reopen the file not in read-only (r) but in read-write mode, then the standard attributes which already exist are not checked again. So if the HDF5 was written with another package, e.g. h5py, then the value might be wrong:

In [14]:
with h5tbx.File(name=h5.hdf_filename, mode='r+') as h5:
    pass # note, that we were not required to pass "data_type" as it was present already!

Note, that a convention can also be enabled only **temporarily** using the context manager syntax:

In [15]:
with h5tbx.use(cv):
    with h5tbx.File(contact_id='id1722') as h5:
        pass

## Importing/Loading an online convention

The intended distribution of convention is via online repositories. The YAML file hence should be uploaded such it is accessible to all users. The `h5RDMtoolbox` currently favors the usage of [Zenodo](https://zenodo.org) repositories. The advantages are long-term storage and assignment of a DOI. However, files accessible via an URL can also be downloaded.

A tutorial convention is published [here](https://zenodo.org/record/8276817). By calling `from_zenodo()` the convention object is created:

In [16]:
cv = h5tbx.convention.from_zenodo(doi='https://zenodo.org/records/10156750')
cv

TypeError: from_zenodo() got an unexpected keyword argument 'doi'

## Effect of enabling a convention

The convention above defined the usage of certain attributes with certain methods. E.g. "data_type" is to be used when a HDF5 file is created. When the convention is enabled, the **signature of the respective methods is changed**. To proof this, let's implement a small function, which prints all parameters of a given function and inspect the effect of the convention in the `__init__` method:

In [ ]:
cv.properties[h5tbx.Dataset]['standard_name']

In [ ]:
import inspect

def print_method_parameters(method):
    print(f'\nParameters for "{method.__name__}":')
    for param in inspect.signature(method).parameters.values():
        if not param.name == 'self':
            if param.name in h5tbx.convention.get_current_convention().methods[h5tbx.File].get('__init__', {}).keys():
                print(f'  - {h5tbx._repr.make_bold(param.name)}')
            else:
                print(f'  - {param.name}')

methods = (h5tbx.File.__init__, h5tbx.Group.create_group, h5tbx.Group.create_dataset)

print('no convention: ')
h5tbx.use(None)
print_method_parameters(h5tbx.File.__init__)

print(f'\n------------\nwith convention {cv.name}: (standard attributes are made bold)')
h5tbx.use(cv)
print_method_parameters(h5tbx.File.__init__)

In [ ]:
h5tbx.use(None)  # fall back to the default convention